In [1]:
# using Jupyter notebooks
# pushing CTRL-c will run the code in a cell
2 + 2

4

# Gentle Introduction to NLP through Word Embeddings

![NLP](images/NLP.png)

# How To Tell If Two Words Are "Similar"?

![distance](images/distance_measures.png)
http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/

# Cosine Similarity

![cos_sim](images/cos_sim.png)
http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/

## calculating dot product
$vector_a = [1,2,3]$ <br>
$vector_b = [4,5,6]$ <br>
$vector_a \cdot vector_b = (1*4) + (2*5) + (3*6) = 4 + 10 + 18 = 32$ 

## Normalizing a Vector

To normalize a vector, we shrink all values so they fall between $0$ and $1$.

$vector_{normalized} = \frac{vector}{\sqrt{vector \cdot vector}}$ <img src="images/normalize.jpg",width=400,height=400>

http://www.wikihow.com/Normalize-a-Vector

<img src="images/unit_circle.png",width=600,height=600>
https://en.wikipedia.org/wiki/Unit_vector

In [2]:
import numpy as np
from nltk.corpus import wordnet
from collections import OrderedDict
from itertools import combinations
import string
from gensim import models

In [3]:
def normalize_vector(vector):
    """
    Normalizes a vector so that all its values are between 0 and 1
    :param vector: a `numpy` vector
    :return: a normalized `numpy` vector
    """
    # norm = np.sqrt(vector.dot(vector))
    # numpy has a built in function
    norm = np.linalg.norm(vector)
    if norm:
        return vector / norm
    else:
        # if norm == 0, then original vector was all 0s
        return vector

In [4]:
vector_3d = np.array([1,2,4])
print("original vector", vector_3d)
print("normalized vector", normalize_vector(vector_3d))
#0.218 is 1/4th of .873 just like 1 is 1/4th of 4

original vector [1 2 4]
normalized vector [ 0.21821789  0.43643578  0.87287156]


## Calculating Cosine Similarity

In [5]:
def cos_sim(vector_one, vector_two):
    """
    Calculate the cosine similarity of two `numpy` vectors
    :param vector_one: a `numpy` vector
    :param vector_two: a `numpy` vector
    :return: A score between 0 and 1
    """
    # ensure that both vectors are already normalized
    vector_one_norm = normalize_vector(vector_one)
    vector_two_norm = normalize_vector(vector_two)
    
    # calculate the dot product between the two normalized vectors
    return vector_one_norm.dot(vector_two_norm)

In [6]:
vector_one = np.array([1,1,1,1,1])
vector_two = np.array([1,1,1,1,2])
vector_three = np.array([1,2,3,4,5])
vector_four = np.array([10,20,30,40,50])

print("cosine similarity of vector_one and vector_two", cos_sim(vector_one, vector_two))
print("cosine similarity of vector_one and vector_three", cos_sim(vector_one, vector_three))
print("cosine similarity of vector_one and vector_four", cos_sim(vector_one, vector_four))

cosine similarity of vector_one and vector_two 0.948683298051
cosine similarity of vector_one and vector_three 0.904534033733
cosine similarity of vector_one and vector_four 0.904534033733


## Measuring the "Similarity" of Words

<img src="images/cos_sim_compare.png",xwidth=1000,height=1000>
https://medium.com/@camrongodbout/creating-a-search-engine-f2f429cab33c#.z7i9w8y5t

<img src="images/vectorize.png", width=1000,height=1000>

### Option 1: One-hot vectors

<img src="images/one_hot.png", width=1000,height=1000>
https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

In [7]:
vocabulary = ['apple', 'banana', 'orange', 'cantaloupe', 'peach']

In [8]:
# generate vocabulary lookup
def build_voc_lookup(list_of_voc):
    """
    Generates a dictionary where the key is the word and the value is its index
    :param list_of_voc: list of vocabulary words
    :return: Dictionary of vocabulary
    """
    lookup_dict = OrderedDict()
    counter = 0
    for word in list_of_voc:
        lookup_dict[word] = counter
        counter+=1
    return lookup_dict

In [9]:
# lookup word
def lookup_word(lookup_dict, word):
    """ 
    Looks up a given word in the vocabulary dictionary, and returns None if word not in vocabulary
    :param lookup_dict: lookup-dictionary built with build_voc_lookup()
    :param word to index
    :return: index of word in vocabulary or None
    """
    if word in lookup_dict:
        return lookup_dict[word]
    else:
        return None

In [10]:
lookup_dict = build_voc_lookup(vocabulary)
print(lookup_word(lookup_dict, 'peach'))
print(lookup_word(lookup_dict, 'hashbrown'))

4
None


In [11]:
# build one-hot vector for word
def make_one_hot(lookup_dict, word):
    """
    Builds a one-hot numpy vector for a word
    :param lookup_dict: lookup-dictionary built with build_voc_lookup()
    :param word: word to convert to one-hot
    :return numpy vector with dimension equal to size of vocabulary
    """
    # get size of vocabulary
    voc_size = len(lookup_dict.items())
    # initialize empty vector of zeros with the size of the vocabulary
    one_hot = np.zeros((voc_size))
    # get index of word (or None if not in vocabulary)
    word_index = lookup_word(lookup_dict, word)
    # make the nth dimension of one-hot (representing the index of word in vocabulary) to 1
    if word_index or word_index == 0:
        one_hot[word_index] = 1
    # if word not in vocabulary, the one-hot will remain zeros
    return one_hot

In [12]:
for word in vocabulary + ['hashbrown', 'Capizzi']:
    print("one-hot vector for '{:>11}'".format(word), make_one_hot(lookup_dict, word))

one-hot vector for '      apple' [ 1.  0.  0.  0.  0.]
one-hot vector for '     banana' [ 0.  1.  0.  0.  0.]
one-hot vector for '     orange' [ 0.  0.  1.  0.  0.]
one-hot vector for ' cantaloupe' [ 0.  0.  0.  1.  0.]
one-hot vector for '      peach' [ 0.  0.  0.  0.  1.]
one-hot vector for '  hashbrown' [ 0.  0.  0.  0.  0.]
one-hot vector for '    Capizzi' [ 0.  0.  0.  0.  0.]


#### The problem with one-hot vectors

In [13]:
# add an OOV word to vocabulary
vocabulary_plus_oov = vocabulary + ["Phoenix"]
# get all combinations
all_combinations = combinations(vocabulary_plus_oov, 2)
# iterate through all combinations and calculate cosine similarity
for (word1, word2) in all_combinations:
    one_hot_word_1 = make_one_hot(lookup_dict, word1)
    one_hot_word_2 = make_one_hot(lookup_dict, word2)
    print("cosine similarity between {} and {}".format(word1, word2), cos_sim(one_hot_word_1, one_hot_word_2))

cosine similarity between apple and banana 0.0
cosine similarity between apple and orange 0.0
cosine similarity between apple and cantaloupe 0.0
cosine similarity between apple and peach 0.0
cosine similarity between apple and Phoenix 0.0
cosine similarity between banana and orange 0.0
cosine similarity between banana and cantaloupe 0.0
cosine similarity between banana and peach 0.0
cosine similarity between banana and Phoenix 0.0
cosine similarity between orange and cantaloupe 0.0
cosine similarity between orange and peach 0.0
cosine similarity between orange and Phoenix 0.0
cosine similarity between cantaloupe and peach 0.0
cosine similarity between cantaloupe and Phoenix 0.0
cosine similarity between peach and Phoenix 0.0


### Option 2: Encode spelling
Following a similar pattern as the one-hot of a word over a vocabulary, let's build word vectors represented by the frequency of the letters present

In [14]:
alphabet = list(string.ascii_lowercase)

In [15]:
# since we don't need to worry about "out-of-vocabulary" now, we can just use alphabet.index([letter])
def lookup_letter(letter):
    return alphabet.index(letter.lower())

In [16]:
print("a", lookup_letter('a'))
print("A", lookup_letter('A'))

a 0
A 0


In [17]:
def make_spelling_vector(word):
    """
    Converts a word into a vector of dimension 26 where each cell contains the count for that letter
    :param word: word to vectorize
    :return: numpy vector of 26 dimensions
    """
    # initialize vector with zeros
    spelling_vector = np.zeros((26))
    # iterate through each letter and update count
    for letter in word:
        if letter in string.ascii_letters:
            letter_index = lookup_letter(letter)
            spelling_vector[letter_index] = spelling_vector[letter_index] + 1
    return spelling_vector

In [18]:
make_spelling_vector("apple")

array([ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [19]:
# what if two words share the same letters?
dog = make_spelling_vector("dog")
god = make_spelling_vector("God")
god == dog

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [20]:
# reset the generator
all_combinations = combinations(vocabulary_plus_oov, 2)
# iterate through all words
for (word1, word2) in all_combinations:
    spelling_vector_1 = make_spelling_vector(word1)
    spelling_vector_2 = make_spelling_vector(word2)
    print("cosine similarity between {} and {}".format(word1, word2), cos_sim(spelling_vector_1, spelling_vector_2))

cosine similarity between apple and banana 0.303045763366
cosine similarity between apple and orange 0.308606699924
cosine similarity between apple and cantaloupe 0.654653670708
cosine similarity between apple and peach 0.676123403783
cosine similarity between apple and Phoenix 0.428571428571
cosine similarity between banana and orange 0.54554472559
cosine similarity between banana and cantaloupe 0.617213399848
cosine similarity between banana and peach 0.3585685828
cosine similarity between banana and Phoenix 0.20203050891
cosine similarity between orange and cantaloupe 0.589255650989
cosine similarity between orange and peach 0.36514837167
cosine similarity between orange and Phoenix 0.462910049886
cosine similarity between cantaloupe and peach 0.645497224368
cosine similarity between cantaloupe and Phoenix 0.436435780472
cosine similarity between peach and Phoenix 0.507092552837


#### We've successfully generated similarity scores!  But...

Do they really reflect anything semantic?  

In other words, does it make sense that **"peach"** and **"Phoenix"** (`cosine similarity = 0.507`) <br>
are **more** similar than **"peach"** and **"orange"** <br>
(`cosine similarity = .365`)?

### Option 3: Word Embeddings
Create a "dense" representation of each word where proximity in vector space represents "similarity".

<img src="images/context.png", width=1000,height=1000>
https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

<img src="images/architecture.png", width=1000,height=1000>
https://arxiv.org/pdf/1301.3781v3.pdf

<img src="images/cbow.png", width=600,height=600>
https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

#### Using the `gensim` package in `python`
https://radimrehurek.com/gensim/models/word2vec.html

In [21]:
# load existing word2vec vectors into gensim

# most frequent 125k words in Gigaword corpus
w2v = models.Word2Vec.load_word2vec_format(fname="Gigaword_pruned_vectors.txt.gz", binary=False)

Pre-trained word embeddings can be loaded into `gensim` in `.txt` or `.txt.gz` format *as long as* the first line identifies (1) the number of words in file and (2) the dimensions of the vector
 
```
199999 200
and -0.065843 -0.133472 0.020263 0.102796 0.003295 0.025878 -0.071714 0.054211 -0.026698 -0.036176 -0.024954 0.042049 -0.165819 -0.067038 0.117293 0.046338 0.012154 0.026929 -0.020248 0.120186 0.081922 0.062471 -0.063391 -0.048321 -0.108106 -0.067974 0.092109 -0.034439 -0.024319 0.008799 -0.099953
...
```

In [22]:
# the first 50 dimensions of the vector for "the"
w2v["the"][0:50]

array([ 0.06338   , -0.146809  ,  0.110004  , -0.01205   , -0.045637  ,
       -0.02224   , -0.045153  ,  0.079144  , -0.027216  , -0.027647  ,
       -0.000434  ,  0.108648  , -0.060456  , -0.129502  ,  0.010897  ,
        0.055499  ,  0.086099  ,  0.055282  ,  0.007365  ,  0.167188  ,
        0.016705  ,  0.0744    , -0.07096   , -0.105974  , -0.095631  ,
        0.006107  ,  0.12862299, -0.033055  , -0.020641  ,  0.024765  ,
       -0.048181  , -0.090195  ,  0.007408  ,  0.073138  ,  0.031994  ,
       -0.014252  ,  0.102764  , -0.081244  ,  0.10513   ,  0.039809  ,
       -0.050727  ,  0.002429  , -0.01506   , -0.085081  , -0.02245   ,
        0.102064  , -0.009099  , -0.092295  , -0.040276  ,  0.148752  ], dtype=float32)

In [24]:
w2v["abcdef"]

KeyError: 'abcdef'

In [28]:
def get_vector(word):
    """
    Returns the word vector for that word or a vector of 0s for out-of-vocabulary
    :param: word: word to lookup in vectors
    :return: vector or vector of zeros
    """
    # determine vector length
    w2v_length = len(w2v["the"])
    # get vector
    if word in w2v:
            return w2v[word]
    else:
        return np.zeros((w2v_length))

In [29]:
get_vector("abcdef")[0:50]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

#### Evaluation of word embeddings

<img src="images/king_queen.png", width=1000,height=1000>
https://arxiv.org/pdf/1301.3781v3.pdf

<img src="images/king_queen_vis.png", width=1000,height=1000>
https://www.aclweb.org/anthology/N/N13/N13-1090.pdf

<img src="images/country_capital.png", width=700,height=700>
https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

<img src="images/eval_1.png", width=1000,height=1000>
https://arxiv.org/pdf/1301.3781v3.pdf

In [25]:
# king - man + woman = queen
sol = w2v.most_similar(
    positive=['king', 'woman'], 
    negative=['man'], 
    topn=5)
print(sol)
print()

# mouse - dollar + dollars = mice
sol_2 = w2v.most_similar(
    positive=['mouse', 'dollars'], 
    negative=['dollar'], 
    topn=5)
print(sol_2)
print()

# brother - uncle + aunt = sister
sol_3 = w2v.most_similar(
    positive=['brother','aunt'],
    negative=['uncle'],
    topn=5)
print(sol_3)

[('queen', 0.6834795475006104), ('monarch', 0.6421915292739868), ('princess', 0.5896612405776978), ('beatrix', 0.5811704993247986), ('prince', 0.5663138031959534)]

[('monkey', 0.4996635317802429), ('hamster', 0.48757076263427734), ('zombie', 0.4652235805988312), ('scoobydoo', 0.44669386744499207), ('piglet', 0.4450364112854004)]

[('sister', 0.8335152268409729), ('daughter', 0.8259485960006714), ('mother', 0.7856060266494751), ('grandmother', 0.7708373069763184), ('sisterinlaw', 0.7601062655448914)]


In [26]:
# find most similar n words to a given word
similar = w2v.similar_by_word("queen", topn=10)
similar

[('monarch', 0.7166919708251953),
 ('princess', 0.7164901494979858),
 ('margrethe', 0.6889792680740356),
 ('beatrix', 0.6878944039344788),
 ('coronation', 0.6789792776107788),
 ('prince', 0.6730599403381348),
 ('wilhelmina', 0.6619384288787842),
 ('mettemarit', 0.6575925946235657),
 ('consort', 0.6492267847061157),
 ('duchess', 0.6444146633148193)]

In [30]:
# find  most similar n words to a given vector
cat_vector = get_vector("cat")
cat_sim = w2v.similar_by_vector(cat_vector, topn=10)
cat_sim

[('cat', 1.0),
 ('dog', 0.8524122834205627),
 ('puppy', 0.7896589040756226),
 ('pug', 0.783139169216156),
 ('critter', 0.7650502324104309),
 ('squirrel', 0.7516598701477051),
 ('feline', 0.7436362504959106),
 ('gerbil', 0.7435644865036011),
 ('monkey', 0.7434572577476501),
 ('hamster', 0.7323285341262817)]

In [31]:
# find which word doesn't match
list_of_words = "breakfast cereal dinner lunch"

doesnt_match = w2v.doesnt_match(list_of_words.split())
doesnt_match

'cereal'

In [32]:
# this approach doesn't handle antonyms well
# "That movie was _______."

w2v.similar_by_word("good", topn=10)

[('bad', 0.7170573472976685),
 ('terrific', 0.7161434888839722),
 ('decent', 0.7018914222717285),
 ('lousy', 0.6984266042709351),
 ('wonderful', 0.6819486618041992),
 ('perfect', 0.6481753587722778),
 ('great', 0.6480209827423096),
 ('nice', 0.6281204223632812),
 ('darn', 0.623289942741394),
 ('fun', 0.6176395416259766)]

#### Bias in Word Embeddings

![king_queen](images/king_queen_2.png)
![programmer_homemaker](images/programmer_homemaker.png)
https://arxiv.org/pdf/1607.06520v1.pdf

<img src="images/gender_bias.png", width=1000,height=1000>
https://arxiv.org/pdf/1607.06520v1.pdf

#### Links to available word embeddings

[The "original" code for `word2vec`, and pre-trained vectors](https://code.google.com/archive/p/word2vec/)

[Stanford's approach to word embeddings, and pre-trained vectors](http://nlp.stanford.edu/projects/glove/)

[A modified approach to word embeddings (feeding dependency tuples to the neural network instead of words), and pre-trained vectors](https://levyomer.wordpress.com/2014/04/25/dependency-based-word-embeddings/)

[Word embeddings from a particular historical period](http://nlp.stanford.edu/projects/histwords/)

## Links to papers

The "original" three papers on `word2vec` by Mikolov:

 - [Efficient Estimation of Word Representations in Vector Space](http://arxiv.org/pdf/1301.3781v3.pdf)

 - [Distributed Representations of Words and Phrases and their Compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

 - [Linguistic Regularities in Continuous Space Word Representations](https://www.aclweb.org/anthology/N/N13/N13-1090.pdf)


[Further analysis of approaches to word embeddings and their hyperparameters](https://transacl.org/ojs/index.php/tacl/article/viewFile/570/124)

[Detailed evaluation of word embeddings](https://arxiv.org/pdf/1608.04207v1.pdf)

[Website for evaluating word embeddings](http://veceval.com/)



## Links to blogs

[A good overview of NLP](https://blog.monkeylearn.com/the-definitive-guide-to-natural-language-processing/)

[Blog post summary of the three "original" papers by Mikolov](https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/)

[Detailed blog post on the application of word embeddings to analogies](https://quomodocumque.wordpress.com/2016/01/15/messing-around-with-word2vec/)

[Appyling word embeddings to computer logs](https://gab41.lab41.org/three-things-we-learned-about-applying-word-vectors-to-computer-logs-c199070f390b#.k2mirf2oa)